# Teams Meeting Fetcher - Complete Workflow

This notebook guides you through the complete setup and testing workflow for the Teams Meeting Fetcher system.

## 📋 Workflow Overview

**Sequence of Operations:**

1. **Verify Setup** - Check environment and permissions
2. **Create Webhook Subscription** - Subscribe to calendar events
3. **Add User to Group** - Add test user to monitored group
4. **Create Test Meeting** - Create meeting with transcript enabled
5. **Test Webhook** - Verify webhook delivery
6. **Poll for Transcript** - Wait for and download transcript

---

## Prerequisites

- ✅ Azure infrastructure deployed (run `terraform apply` in `iac/azure/`)
- ✅ Admin consent granted for Graph API permissions
- ✅ `.env.local.azure` configured with all values
- ✅ Python dependencies installed: `pip install msal requests python-dotenv`

---

## Step 1: Verify Setup ✅

First, verify that your environment is correctly configured and you have the necessary permissions.

In [ ]:
# Run verification script
%run scripts/graph/01-verify-setup.py

## Step 2: Create Webhook Subscription 🔔

Subscribe to calendar event changes for your test user.

In [ ]:
# Use the subscription script to create a webhook
# This will prompt you for user email and options
import sys
sys.path.append('scripts/graph')

from auth_helper import get_graph_headers, get_config
import requests
from datetime import datetime, timedelta

# Configuration
TEST_USER_EMAIL = "testuser@yourdomain.com"  # Replace with your test user email

def create_subscription_for_user(user_email):
    config = get_config()
    webhook_url = config['webhook_url']
    
    headers = get_graph_headers()
    url = "https://graph.microsoft.com/v1.0/subscriptions"
    
    expiration = datetime.utcnow() + timedelta(hours=24)
    
    payload = {
        "changeType": "created,updated",
        "notificationUrl": webhook_url,
        "resource": f"users/{user_email}/events",
        "expirationDateTime": expiration.strftime("%Y-%m-%dT%H:%M:%S.0000000Z"),
        "clientState": config.get('webhook_secret', 'test-state')[:255]
    }
    
    response = requests.post(url, headers=headers, json=payload, timeout=30)
    
    if response.status_code == 201:
        subscription = response.json()
        print("✅ Subscription created!")
        print(f"   ID: {subscription['id']}")
        print(f"   Expires: {subscription['expirationDateTime']}")
        return subscription
    else:
        print(f"❌ Failed: {response.status_code}")
        print(response.text)
        return None

# Create subscription
subscription = create_subscription_for_user(TEST_USER_EMAIL)
if subscription:
    SUBSCRIPTION_ID = subscription['id']
    print(f"\n💾 Save this subscription ID: {SUBSCRIPTION_ID}")

## Step 3: Add Test User to Monitored Group 👥

Ensure your test user is in the target Entra group so their meetings are tracked.

In [ ]:
# Add user to target group
from auth_helper import get_config
import requests

def add_user_to_group(user_email):
    config = get_config()
    group_id = config['group_id']
    headers = get_graph_headers()
    
    # Get user ID
    user_url = f"https://graph.microsoft.com/v1.0/users/{user_email}"
    user_response = requests.get(user_url, headers=headers, timeout=10)
    
    if user_response.status_code == 200:
        user_id = user_response.json()['id']
        
        # Add to group
        group_url = f"https://graph.microsoft.com/v1.0/groups/{group_id}/members/$ref"
        payload = {"@odata.id": f"https://graph.microsoft.com/v1.0/users/{user_id}"}
        
        response = requests.post(group_url, headers=headers, json=payload, timeout=10)
        
        if response.status_code == 204:
            print(f"✅ User {user_email} added to group")
            return True
        elif 'already exist' in response.text:
            print(f"✅ User already in group")
            return True
        else:
            print(f"❌ Error: {response.status_code}")
            return False
    else:
        print(f"❌ User not found: {user_email}")
        return False

add_user_to_group(TEST_USER_EMAIL)

## Step 4: Create Test Meeting with Transcript Enabled 📅

Create a Teams meeting with automatic transcription enabled.

In [ ]:
# Create Teams meeting with transcript enabled
from datetime import datetime, timedelta

def create_meeting_with_transcript(user_email):
    headers = get_graph_headers()
    url = f"https://graph.microsoft.com/v1.0/users/{user_email}/events"
    
    start_time = datetime.utcnow() + timedelta(hours=1)
    end_time = start_time + timedelta(minutes=60)
    
    payload = {
        "subject": "Test Teams Meeting - Transcript Enabled",
        "body": {
            "contentType": "HTML",
            "content": "<b>Test meeting with transcription enabled</b><br>Join and record to test workflow."
        },
        "start": {
            "dateTime": start_time.strftime("%Y-%m-%dT%H:%M:%S"),
            "timeZone": "UTC"
        },
        "end": {
            "dateTime": end_time.strftime("%Y-%m-%dT%H:%M:%S"),
            "timeZone": "UTC"
        },
        "isOnlineMeeting": True,
        "onlineMeetingProvider": "teamsForBusiness"
    }
    
    response = requests.post(url, headers=headers, json=payload, timeout=30)
    
    if response.status_code == 201:
        event = response.json()
        print("✅ Meeting created!")
        print(f"   Event ID: {event['id']}")
        print(f"   Start: {start_time.strftime('%Y-%m-%d %H:%M UTC')}")
        
        if 'onlineMeeting' in event:
            meeting = event['onlineMeeting']
            print(f"   Join URL: {meeting.get('joinUrl', 'N/A')}")
            
            # Try to enable transcript
            if meeting.get('id'):
                enable_transcript_url = f"https://graph.microsoft.com/v1.0/communications/onlineMeetings/{meeting['id']}"
                transcript_payload = {
                    "recordAutomatically": True,
                    "allowTranscription": True
                }
                
                transcript_response = requests.patch(enable_transcript_url, headers=headers, json=transcript_payload, timeout=10)
                
                if transcript_response.status_code == 200:
                    print("   ✅ Transcript enabled!")
                else:
                    print(f"   ⚠️  Could not enable transcript (may require additional permissions)")
        
        return event
    else:
        print(f"❌ Failed: {response.status_code}")
        print(response.text)
        return None

# Create the meeting
meeting_event = create_meeting_with_transcript(TEST_USER_EMAIL)
if meeting_event:
    MEETING_ID = meeting_event['id']
    ONLINE_MEETING_ID = meeting_event.get('onlineMeeting', {}).get('id')
    print(f"\n💾 Save these IDs:")
    print(f"   Event ID: {MEETING_ID}")
    print(f"   Online Meeting ID: {ONLINE_MEETING_ID}")

## Step 5: Test Webhook Delivery 🔌

Verify that webhooks are being received and processed correctly.

In [ ]:
# Test webhook with a mock payload
def test_webhook():
    config = get_config()
    webhook_url = config['webhook_url']
    webhook_secret = config.get('webhook_secret')
    
    test_payload = {
        "value": [{
            "subscriptionId": SUBSCRIPTION_ID if 'SUBSCRIPTION_ID' in globals() else "test-id",
            "changeType": "created",
            "resource": f"users/{TEST_USER_EMAIL}/events/{MEETING_ID if 'MEETING_ID' in globals() else 'test-123'}",
            "resourceData": {
                "@odata.type": "#Microsoft.Graph.event",
                "id": MEETING_ID if 'MEETING_ID' in globals() else "test-123"
            },
            "clientState": webhook_secret
        }]
    }
    
    headers = {'Content-Type': 'application/json'}
    if webhook_secret:
        headers['Authorization'] = f'Bearer {webhook_secret}'
    
    response = requests.post(webhook_url, json=test_payload, headers=headers, timeout=10)
    
    print(f"Webhook Response: {response.status_code}")
    print(f"Body: {response.text[:200]}")
    
    return response.status_code == 200

# Test the webhook
test_webhook()

## Step 6: Poll for Transcript (After Meeting) 📝

⚠️ **Important**: This step should be run AFTER you:
1. Join the meeting
2. Start recording in Teams
3. Speak to generate some transcript content
4. Stop recording and end the meeting

Transcripts typically become available 5-10 minutes after the meeting ends.

In [ ]:
# Poll for transcription
import time

def poll_for_transcript(user_email, meeting_id, max_attempts=10, delay_seconds=30):
    headers = get_graph_headers()
    url = f"https://graph.microsoft.com/v1.0/users/{user_email}/onlineMeetings/{meeting_id}/transcripts"
    
    for attempt in range(1, max_attempts + 1):
        print(f"[{datetime.now().strftime('%H:%M:%S')}] Attempt {attempt}/{max_attempts}...")
        
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            transcripts = response.json().get('value', [])
            
            if transcripts:
                print(f"\n✅ Found {len(transcripts)} transcript(s)!")
                
                for idx, transcript in enumerate(transcripts, 1):
                    print(f"\nTranscript {idx}:")
                    print(f"  ID: {transcript['id']}")
                    print(f"  Created: {transcript.get('createdDateTime', 'N/A')}")
                    
                    # Download transcript content
                    content_url = f"https://graph.microsoft.com/v1.0/users/{user_email}/onlineMeetings/{meeting_id}/transcripts/{transcript['id']}/content"
                    content_response = requests.get(content_url, headers=headers, timeout=30)
                    
                    if content_response.status_code == 200:
                        filename = f"transcript_{meeting_id}_{idx}.vtt"
                        with open(filename, 'w', encoding='utf-8') as f:
                            f.write(content_response.text)
                        print(f"  ✅ Saved to: {filename}")
                        
                        # Display first few lines
                        lines = content_response.text.split('\n')[:10]
                        print(f"\n  Preview:")
                        for line in lines:
                            print(f"    {line}")
                
                return transcripts
        
        if attempt < max_attempts:
            print(f"  No transcript yet, waiting {delay_seconds}s...\n")
            time.sleep(delay_seconds)
    
    print("\n⚠️ No transcript found after maximum attempts")
    return None

# Poll for transcript (only run after meeting is recorded)
if 'ONLINE_MEETING_ID' in globals() and ONLINE_MEETING_ID:
    print("Starting transcript polling...")
    print("Make sure you've already:")
    print("- Joined the meeting")
    print("- Started recording")
    print("- Spoke to generate transcript")
    print("- Ended the recording\n")
    
    # Uncomment the line below when ready to poll
    # transcripts = poll_for_transcript(TEST_USER_EMAIL, ONLINE_MEETING_ID, max_attempts=10, delay_seconds=30)
    print("⚠️ Uncomment the poll_for_transcript call above when ready to poll")
else:
    print("❌ Meeting not created yet. Run Step 4 first.")

---

## 📊 Workflow Summary

### Execution Sequence

1. ✅ **Verify Setup** (`01-verify-setup.py`)
   - Check environment variables
   - Verify Graph API permissions
   - Confirm group exists
   - Test webhook endpoint reachability

2. 🔔 **Create Webhook Subscription** (`02-create-webhook-subscription.py`)
   - Subscribe to user calendar events
   - Store subscription ID for renewal

3. 👥 **Add User to Group** (`05-manage-group.py`)
   - Add test user to monitored Entra group
   - Verify membership

4. 📅 **Create Test Meeting** (`03-create-test-meeting.py`)
   - Create Teams meeting with transcript enabled
   - Save meeting IDs

5. 🔌 **Test Webhook** (`06-test-webhook.py`)
   - Send mock notification
   - Verify webhook processing

6. 📝 **Poll for Transcript** (`04-poll-transcription.py`)
   - Wait for transcript after meeting
   - Download transcript content

---

### Script Files Reference

All scripts are located in `scripts/graph/`:

- `auth_helper.py` - Shared authentication logic
- `01-verify-setup.py` - Environment verification
- `02-create-webhook-subscription.py` - Subscription management
- `03-create-test-meeting.py` - Meeting creation
- `04-poll-transcription.py` - Transcript polling
- `05-manage-group.py` - Group management
- `06-test-webhook.py` - Webhook testing

---

### Next Steps

After completing this workflow:

1. **Set up monitoring** - Monitor webhook delivery in CloudWatch (AWS) or App Insights (Azure)
2. **Test real meetings** - Schedule actual meetings and verify workflow
3. **Implement backend service** - Create Azure service to process webhooks
4. **Build UI dashboard** - Display meetings and transcripts
5. **Set up subscription renewal** - Automate 29-day subscription renewal

---

### Troubleshooting

**No webhooks received?**
- Check webhook URL is publicly accessible
- Verify subscription is active (`02-create-webhook-subscription.py` list)
- Check CloudWatch logs (AWS) or Application Insights (Azure)

**No transcript available?**
- Ensure meeting was recorded (manual step in Teams)
- Wait 5-10 minutes after meeting ends
- Verify `allowTranscription` was set on meeting
- Check user has Teams license with transcription enabled

**Permission errors?**
- Grant admin consent in Azure Portal
- Verify service principal has required permissions
- Check RBAC assignments in Azure

---

## 🎉 Success!

You now have a complete workflow for:
- Creating Teams meetings with transcription
- Receiving webhook notifications
- Downloading meeting transcripts

Proceed to implement the backend service to automate this workflow!